In [1]:

from pyarma import *

A = mat(4, 5, fill.randu)
B = mat(4, 5, fill.randu)
  
C = A*B.t()

C.print("C:")

C:
   0.8635   0.6455   1.2309   0.3664
   0.9823   0.7291   1.3008   0.7792
   1.0882   1.0004   1.6399   0.6935
   1.4101   0.9859   2.0146   0.7092


In [17]:
A = mat(3, 2, fill.randu);

print(A)

A.reshape(5,4)

A

<pyarma.pyarma.mat object at 00000236C32F9850>
[matrix size: 3x2]
   0.2358   0.9341
   0.3640   0.9989
   0.3630   0.2010



<pyarma.pyarma.mat object at 00000236C32F9850>
[matrix size: 5x4]
   0.2358   0.2010        0        0
   0.3640        0        0        0
   0.3630        0        0        0
   0.9341        0        0        0
   0.9989        0        0        0

In [29]:
A = mat(3, 2, fill.randu);

print(A)

A.resize(5,5);

print(A)

A[:,1:2]

<pyarma.pyarma.mat object at 00000236C32F9B90>
[matrix size: 3x2]
   0.1295   0.5738
   0.3725   0.5908
   0.9607   0.5239

<pyarma.pyarma.mat object at 00000236C32F9B90>
[matrix size: 5x5]
   0.1295   0.5738        0        0        0
   0.3725   0.5908        0        0        0
   0.9607   0.5239        0        0        0
        0        0        0        0        0
        0        0        0        0        0



<pyarma.pyarma.__subview_mat object at 0000023703792990>
[matrix size: 5x2]
   0.5738        0
   0.5908        0
   0.5239        0
        0        0
        0        0

In [31]:
import numpy as np

In [35]:
np.array(np.random.uniform(0,1,10)).reshape((2,5))[0:2]

array([[0.37136221, 0.41697142, 0.50718125, 0.70331662, 0.56649942],
       [0.62159117, 0.69808115, 0.47440331, 0.55340851, 0.44149835]])

In [37]:
[1,2,3,4,5][1:3]

[2, 3]

In [39]:
x = 0
for i in range(100):
    x += .1
x

9.99999999999998

In [46]:
np.log(randu())

-1.00142163592215

In [48]:
import pandas as pd

In [56]:
data = pd.read_csv('data_prob_90.csv', header=0, delim_whitespace=False)
data_name = data.columns[1:]



data = mat(np.array(data.iloc[:, 1:]))

In [63]:
randu(1,2,8)

<pyarma.pyarma.cube object at 0000023703E26010>
[cube size: 1x2x8]
[cube slice: 0]
   0.4281   0.7754

[cube slice: 1]
   0.0152   0.5220

[cube slice: ...]

[cube slice: 7]
   0.5278   0.2344

In [64]:
from pyarma import *
from math import *

In [25]:
def onepl_lsrm_cont_missing(data,

                            ndim,
                            niter,
                            nburn,
                            nthin,
                            nprint,

                            jump_beta,
                            jump_theta,
                            jump_gamma,
                            jump_z,
                            jump_w,

                            pr_mean_beta,
                            pr_sd_beta,
                            pr_a_th_sigma,
                            pr_b_th_sigma,
                            pr_mean_theta,

                            pr_a_sigma,
                            pr_b_sigma,
                            pr_mean_gamma,
                            pr_sd_gamma,

                            missing):
    ##############################################################################

    def update_function(beta, theta, gamma):
        return (- pow((data[k, i] - beta[i] - theta[k] + gamma * dist[k, i]), 2) /
                (2 * pow(pr_sd, 2))
                )

    def rejection_algorithm(ratio):
        if ratio > 0.0:
            accept = 1
        else:
            un = randu()
            if np.log(un) < ratio:
                accept = 1
            else:
                accept = 0

        return accept

    ##############################################################################

    #nsample = data.shape[0]
    #nitem = data.shape[1]
    
    nsample = data.n_rows
    nitem = data.n_cols

    pr_mean_z = 0.0
    pr_mean_w = 0.0

    pr_sd_z = 1.0
    pr_sd_w = 1.0
    pr_sd = 1.0
    pr_sd_theta = 1.0
    
    """
    oldbeta = np.randuom.uniform(size=nitem)
    oldtheta = np.random.uniform(size=nsample)
    oldz = np.random.uniform(size=(nsample, ndim))
    oldw = np.random.uniform(size=(nitem, ndim))
    """
    
    oldbeta = randu(nitem)
    oldtheta = randu(nsample)
    oldz = randu(nsample, ndim)
    oldw = randu(nitem, ndim)

    # oldbeta = np.random.uniform(nitem)
    # oldtheta = np.random.uniform(nsample)
    # oldz = np.random.rand(nsample, ndim)
    # oldw = np.random.rand(nitem, ndim)

    oldbeta = oldbeta * 4.0 - 2.0
    oldtheta = oldtheta * 4.0 - 2.0
    oldz = oldz * 2.0 - 1.0
    oldw = oldw * 2.0 - 1.0

    newbeta = (oldbeta)
    newtheta = (oldtheta)
    newz = (oldz)
    neww = (oldw)

    ##############################################################################

    oldgamma = 1
    newgamma = 1  # gamma1 = log(gamma)

    samp_beta = zeros((niter - nburn) // nthin, nitem)
    samp_theta = zeros((niter - nburn) // nthin, nsample)

    samp_z = zeros((niter - nburn) // nthin, nsample, ndim)
    samp_w = zeros((niter - nburn) // nthin, nitem, ndim)

    samp_sd_theta = zeros((niter - nburn) // nthin)
    samp_sd = zeros((niter - nburn) // nthin)
    samp_mle = zeros((niter - nburn) // nthin)
    samp_gamma = zeros((niter - nburn) // nthin)

    accept_beta = zeros(nitem)
    accept_theta = zeros(nsample)
    accept_z = zeros(nsample)
    accept_w = zeros(nitem)

    accept_gamma = 0
    accept = 0
    count = 0

    dist = zeros(nsample, nitem)

    old_dist_k = zeros(nitem)
    new_dist_k = zeros(nitem)
    old_dist_i = zeros(nsample)
    new_dist_i = zeros(nsample)

    ##############################################################################

    for iter in range(niter):
        # dist(j,i) is distance of z_j and w_i

        dist.zeros()  # 매 이터레이션마다 거리 매트릭스를 0으로 리셋

        for i in range(nitem):
            for k in range(nsample):
                dist_temp = 0.0
                for j in range(ndim):
                    dist_temp += pow((oldz[k, j] - oldw[i, j]), 2.0)
                    dist[k, i] = sqrt(dist_temp)

        # beta update
        for i in range(nitem):
            # TODO 컬럼부터 갱신하는 이유가 있는건가?

            #newbeta[i] = np.random.normal(oldbeta[i], jump_beta, 1)
            newbeta[i] = oldbeta[i] + jump_beta*randn()
            
            old_like_beta = 0.0
            new_like_beta = 0.0

            for k in range(nsample):
                if data[k, i] != missing:
                    new_like_beta += update_function(newbeta, oldtheta, oldgamma)
                    old_like_beta += update_function(oldbeta, oldtheta, oldgamma)
                    

            #num = (new_like_beta +
            #       scipy.stats.norm.logpdf(newbeta[i], pr_mean_beta, pr_sd_beta))
            
            num = (new_like_beta +
                   log_normpdf(newbeta[i], pr_mean_beta, pr_sd_beta))
            den = (old_like_beta +
                   log_normpdf(oldbeta[i], pr_mean_beta, pr_sd_beta))
            ratio = num - den

            accept = rejection_algorithm(ratio)


            if accept == 1:
                oldbeta[i] = newbeta[i]
                accept_beta[i] += 1.0 / (niter * 1.0)

            else:
                newbeta[i] = oldbeta[i]

        # theta update
        for k in range(nsample):
            #newtheta[k] = np.random.normal(oldtheta[k], jump_theta, 1)
            newtheta[k] = oldtheta[k] + jump_theta * randn()
            
            old_like_theta = 0.0
            new_like_theta = 0.0

            for i in range(nitem):
                if data[k, i] != missing:
                    new_like_theta += update_function(oldbeta, newtheta, oldgamma)
                    old_like_theta += update_function(oldbeta, oldtheta, oldgamma)

            num = (new_like_theta +
                   log_normpdf(newtheta[k], pr_mean_theta, pr_sd_theta))
            den = (old_like_theta +
                   log_normpdf(oldtheta[k], pr_mean_theta, pr_sd_theta))
            ratio = num - den

            accept = rejection_algorithm(ratio)

            if accept == 1:
                oldtheta[k] = newtheta[k]
                accept_theta[k] += 1.0 / (niter * 1.0)
            else:
                newtheta[k] = oldtheta[k]

        # gamma(log(gamma)) update

        #newgamma = np.random.lognormal(log(oldgamma), jump_gamma, 1)
        
        newgamma = exp(log(oldgamma) + jump_gamma*randn())
        old_like_gamma = new_like_gamma = 0.0

        for k in range(nsample):
            for i in range(nitem):
                if data[k, i] != missing:
                    new_like_gamma += update_function(oldbeta, newtheta, newgamma)
                    old_like_gamma += update_function(oldbeta, newtheta, oldgamma)

        num = (new_like_gamma +
               log_normpdf(log(oldgamma), log(newgamma), jump_gamma) + 
               log_normpdf(log(newgamma), pr_mean_gamma, pr_sd_gamma)
               #scipy.stats.lognorm.logpdf(oldgamma, s=jump_gamma, loc=0, scale=exp(log(newgamma))) +
               #scipy.stats.lognorm.logpdf(newgamma, s=pr_sd_gamma, loc=0, scale=exp(pr_mean_gamma))
               )
        den = (old_like_gamma +
               log_normpdf(log(newgamma), log(oldgamma), jump_gamma) + 
               log_normpdf(log(oldgamma), pr_mean_gamma, pr_sd_gamma)
               #scipy.stats.lognorm.logpdf(newgamma, s=jump_gamma, loc=0, scale=exp(log(oldgamma))) +
               #scipy.stats.lognorm.logpdf(oldgamma, s=pr_sd_gamma, loc=0, scale=exp(pr_mean_gamma))
               )
        ratio = num - den

        accept = rejection_algorithm(ratio)

        if accept == 1:
            oldgamma = newgamma
            accept_gamma += 1.0 / (niter * 1.0)
        else:
            newgamma = oldgamma

        # zj update

        for k in range(nsample):
            for j in range(ndim):
                newz[k, j] = oldz[k, j] + jump_z*randn()
            old_like_z = 0.0
            new_like_z = 0.0

            # calculate distance of oldw and newz

            for i in range(nitem):
                dist_old_temp = 0.0
                dist_new_temp = 0.0
                
                for j in range(ndim):
                    dist_new_temp += pow((newz[k, j] - oldw[i, j]), 2.0)
                    dist_old_temp += pow((oldz[k, j] - oldw[i, j]), 2.0)
                new_dist_k[i] = sqrt(dist_new_temp)
                old_dist_k[i] = sqrt(dist_old_temp)

            # calculate likelihood

            for i in range(nitem):
                if data[k, i] != missing:
                    new_like_z += (- pow((data[k, i] - oldbeta[i] - oldtheta[k] + oldgamma * new_dist_k[i]), 2) /
                                   (2 * pow(pr_sd, 2))
                                   )
                    old_like_z += (- pow((data[k, i] - oldbeta[i] - oldtheta[k] + oldgamma * old_dist_k[i]), 2) /
                                   (2 * pow(pr_sd, 2))
                                   )

            num = den = 0.0

            for j in range(ndim):
                num += log_normpdf(newz[k, j], pr_mean_z, pr_sd_z)
                #num += scipy.stats.norm.logpdf(newz[k, j], pr_mean_z, pr_sd_z)
                den += log_normpdf(oldz[k, j], pr_mean_z, pr_sd_z)

            # Rprintf("%.3f %.3f %.3f %.3f\n", num, den, new_like_z, old_like_z)
            # arma::dvec newzz = dmvnorm(newz.cols(2*j,2*j+1),pr_mean_z,pr_cov_z,TRUE)
            # arma::dvec oldzz = dmvnorm(oldz.cols(2*j,2*j+1),pr_mean_z,pr_cov_z,TRUE)

            num += new_like_z
            den += old_like_z
            ratio = num - den

            accept = rejection_algorithm(ratio)

            if accept == 1:
                for j in range(ndim):
                    oldz[k, j] = newz[k, j]
                accept_z[k] += 1.0 / (niter * 1.0)

            else:
                for j in range(ndim):
                    newz[k, j] = oldz[k, j]

        
        
        #  wi update
        for i in range(nitem):
            for j in range(ndim):
                neww[i, j] = oldw[i, j] + jump_w*randn()
                #neww[i, j] = np.random.normal(oldw[i, j], jump_w, 1)
            old_like_w = 0.0
            new_like_w = 0.0

            # calculate distance of neww and oldz

            for k in range(nsample):
                dist_old_temp = 0.0
                dist_new_temp = 0.0
                
                for j in range(ndim):
                    dist_new_temp += pow((oldz[k, j] - neww[i, j]), 2.0)  # TODO: Why Old - New?
                    dist_old_temp += pow((oldz[k, j] - oldw[i, j]), 2.0)
                new_dist_i[k] = sqrt(dist_new_temp)
                old_dist_i[k] = sqrt(dist_old_temp)

            # calculate likelihood

            for k in range(nsample):
                if data[k, i] != missing:
                    new_like_w += (- pow((data[k, i] - oldbeta[i] - oldtheta[k] + oldgamma * new_dist_i[k]), 2) /
                                   (2 * pow(pr_sd, 2))
                                   )
                    old_like_w += (- pow((data[k, i] - oldbeta[i] - oldtheta[k] + oldgamma * old_dist_i[k]), 2) /
                                   (2 * pow(pr_sd, 2))
                                   )

            num = den = 0.0

            for j in range(ndim):
                num += log_normpdf(neww[i, j], pr_mean_w, pr_sd_w)
                #num += scipy.stats.norm.logpdf(neww[i, j], pr_mean_w, pr_sd_w)
                den += log_normpdf(oldw[i, j], pr_mean_w, pr_sd_w)

            num += new_like_w
            den += old_like_w
            ratio = num - den

            accept = rejection_algorithm(ratio)

            if accept == 1:
                for j in range(ndim):
                    oldw[i, j] = neww[i, j]
                accept_w[i] += 1.0 / (niter * 1.0)
            else:
                for j in range(ndim):
                    neww[i, j] = oldw[i, j]

        # sigma_theta update with gibbs

        
        post_a_th_sigma = pr_a_th_sigma * 2 + nsample
        post_b_th_sigma = pr_b_th_sigma

        for j in range(nsample):  # TODO: 여기 j인데 nsample인거 맞음?
            post_b_th_sigma += pow((oldtheta[j] - pr_mean_theta), 2.0)
        pr_sd_theta = sqrt(2 * post_b_th_sigma * (1.0/chi2rnd(post_a_th_sigma)))
                                                  #sum(square(randn(post_a_th_sigma//1)))))
                                                  #np.random.chisquare(post_a_th_sigma)))
                                                  
                                                  
        
        

        # dist(j,i) is distance of z_j and w_i

        dist.zeros()

        for i in range(nitem):
            for k in range(nsample):
                dist_temp = 0.0
                for j in range(ndim):
                    dist_temp += pow((oldz[k, j] - oldw[i, j]), 2.0)
                dist[k, i] = sqrt(dist_temp)

        # sigma update with gibbs

        post_a_sigma = pr_a_sigma * 2 + nsample * nitem
        post_b_sigma = pr_b_sigma

        for j in range(nsample):  # TODO: 여기 j인데 nsample인거 맞음?
            for i in range(nitem):
                post_b_sigma += pow((data[j, i] - oldbeta[i] - oldtheta[j] + oldgamma * dist[j, i]), 2.0) / 2

        pr_sd = sqrt(2 * post_b_sigma * (1.0/chi2rnd(post_a_th_sigma))) #post_a_th_sigma//
        
                                         #np.random.chisquare(post_a_sigma)))
                                         
                                         
                                         
                                         

        # burn, thin

        if iter >= nburn and iter % nthin == 0:
            for i in range(nitem):
                samp_beta[count, i] = oldbeta[i]
            for k in range(nsample):
                samp_theta[count, k] = oldtheta[k]
            for i in range(nitem):
                for j in range(ndim):
                    samp_w[count, i, j] = oldw[i, j]
            for k in range(nsample):
                for j in range(ndim):
                    samp_z[count, k, j] = oldz[k, j]

            samp_gamma[count] = oldgamma
            samp_sd_theta[count] = pr_sd_theta
            samp_sd[count] = pr_sd

            mle = 0.0

            for i in range(nitem):
                mle += log_normpdf(oldbeta[i], pr_mean_beta, pr_sd_beta)
                #mle += scipy.stats.norm.logpdf(oldbeta[i], pr_mean_beta, pr_sd_beta)
            for k in range(nsample):
                mle += log_normpdf(oldtheta[k], pr_mean_theta, pr_sd_theta)
            for i in range(nitem):
                for j in range(ndim):
                    mle += log_normpdf(oldw[i, j], pr_mean_w, pr_sd_w)
            for k in range(nsample):
                for j in range(ndim):
                    mle += log_normpdf(oldz[k, j], pr_mean_z, pr_sd_z)
            for k in range(nsample):
                for i in range(nitem):
                    mle += (- pow((data[k, i] - oldbeta[i] - oldtheta[k] + oldgamma * dist[k, i]), 2)
                            / (2 * pow(pr_sd, 2)))

            
            mle += log_normpdf(log(oldgamma), pr_mean_gamma, pr_sd_gamma)
            #mle += scipy.stats.lognorm.logpdf(oldgamma, s=pr_sd_gamma, loc=0, scale=exp(pr_mean_gamma))
            samp_mle[count] = mle

            count += 1

        if iter % nprint == 0:
            print("Iteration: ", iter)
            print("count", count)
            for i in range(nitem):
                print("nitem: ", i, " with ", oldbeta[i])
            print("oldgamma: ", oldgamma, "     pr_sd_theta: ", pr_sd_theta)

    return {"beta": samp_beta,
            "theta": samp_theta,
            "z": samp_z,
            "w": samp_w,
            "gamma": samp_gamma,
            "sigma_theta": samp_sd_theta,
            "sigma": samp_sd,
            "map": samp_mle,
            "accept_beta": accept_beta,
            "accept_theta": accept_theta,
            "accept_z": accept_z,
            "accept_w": accept_w,
            "accept_gamma": accept_gamma}

<pyarma.pyarma.mat object at 0000023703FE9840>
[matrix size: 1x1]
   2.4195

In [8]:
from math import *

import numpy as np
import pandas as pd

import scipy

from math import *
import numpy as np
import scipy.stats

from copy import *

from sklearn import datasets
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale

In [9]:
data = pd.read_csv('data_prob_90.csv', header=0, delim_whitespace=False)
data_name = data.columns[1:]
data = np.array(data)

In [10]:
data_word = pd.DataFrame(data[:, 0]) # 데이터 인풋. 인풋된 데이터는 data. input 되는 것은 논문 더미들.

data = np.delete(data, 0, 1) # 0번째 column은 data_word. 뽑아내고 나머지 데이타 보존.

data = np.nan_to_num(data, copy=True, nan=99) # na인 cell은 값을 99로 설정.

In [11]:
data = np.array(data, dtype=float)

In [12]:
data_m = data # 데이터 타입 매트릭스로 바뀐 데이터. 파이썬에선 pd.dataframe 사용

logit_x = np.log(data_m / (1 - data_m)) # 매트릭스 데이터를 logit化

In [13]:
from sklearn.preprocessing import MinMaxScaler

data_m = MinMaxScaler().fit_transform(data_m)

In [14]:



# <editor-fold desc="covid_20 parameters">

"""
ndim     = 2 
niter    = 55000
nburn    = 5000
nthin    = 5
nprint   = 5000
"""

ndim = 2 #차원?
niter = 200
nburn = 0
nthin = 5 #thining lapse
nprint = 100



"""
모든 topic의 분포는 theta ~ 디리클레 알파를 따름

biterm 총체 B에서 biterm b를 뽑으면, 이 biterm이 어느 topic z에 속할지는 z ~ 다항분포 세타

topic z의 topic-word 분포는 phi_z ~ 디리클레 베타를 따름
topic = z, word = w. z가 정해졌을 때 이로부터 각 단어가 가지는 확률을 모아서 set으로 한것이 phi_z.

골라진 topic에 대응하는 topic-word 분포로부터 단어 2개가 골라질 확률은 w_i, w_j ~ 다항(phi_z)를 따름
"""

jump_beta = 0.3
jump_theta = 1.0
jump_w = 0.06
jump_z = 0.50
jump_gamma = 0.01

pr_mean_beta = 0
pr_sd_beta = 1
pr_mean_theta = 0
pr_sd_theta = 1
pr_mean_gamma = 0.0
pr_sd_gamma = 1.0
pr_a_sigma = 0.001
pr_b_sigma = 0.001
pr_a_th_sigma = 0.001
pr_b_th_sigma = 0.001
# </editor-fold> #TODO 뭐지? 패러미터 스태티스틱?
#TODO 뭐지? 패러미터 스태티스틱?

----

In [15]:
data = logit_x

In [16]:
#from pyarma import *

In [17]:
missing = 99

In [18]:
import copy

In [19]:
post_a_th_sigma

NameError: name 'post_a_th_sigma' is not defined

In [26]:
logit_x = mat(logit_x)

output = onepl_lsrm_cont_missing(logit_x,
                                 ndim, niter, nburn, nthin, nprint,
                                 jump_beta, jump_theta, jump_gamma, jump_z, jump_w,
                                 pr_mean_beta, pr_sd_beta, pr_a_th_sigma, pr_b_th_sigma, pr_mean_theta,
                                 pr_a_sigma, pr_b_sigma, pr_mean_gamma, pr_sd_gamma,
                                 99)

Iteration:  0
count 1
nitem:  0  with  -1.014847381525794
nitem:  1  with  -1.2065598313876296
nitem:  2  with  0.30897432558000626
nitem:  3  with  -1.7232494559262002
nitem:  4  with  -1.6222650365270164
nitem:  5  with  0.40473846789246415
nitem:  6  with  -0.015435949255703574
nitem:  7  with  1.9703420061905978
nitem:  8  with  1.3548053850468464
nitem:  9  with  -0.7999437199286434
nitem:  10  with  -0.13129407143117877
nitem:  11  with  -1.7966029416158933
nitem:  12  with  -1.0469756687001777
nitem:  13  with  -0.9086248551558364
nitem:  14  with  -1.01486204750209
nitem:  15  with  -0.05113743969888285
nitem:  16  with  -0.24274030342231928
nitem:  17  with  -0.9637447595568059
nitem:  18  with  1.681029374502445
nitem:  19  with  1.1133749072635486
oldgamma:  1.0071545306075054      pr_sd_theta:  2.154734404943964
Iteration:  100
count 21
nitem:  0  with  -6.479237327723866
nitem:  1  with  -0.37869916702831546
nitem:  2  with  -0.9549865213994746
nitem:  3  with  0.155152253

In [4]:
from pyarma import *

In [27]:
output

{'beta': <pyarma.pyarma.mat object at 000001E392E13490>
 [matrix size: 40x20]
   -1.0148  -1.2066   0.3090   ...   1.1134
   -0.8802  -1.7158   0.1287   ...   0.1731
   -0.9720  -1.2216   1.0414   ...   0.5790
         :        :        :   ...        :
   -7.4135   0.0575   3.0558   ...   3.9777,
 'theta': <pyarma.pyarma.mat object at 000001E392E13F20>
 [matrix size: 40x548]
    -0.3600    0.4527   -1.7514   ...    0.1829
    -3.7228    4.5572   -2.3890   ...   -4.1766
    -5.4823    2.9234   -2.4197   ...   -3.7259
          :         :         :   ...         :
   -24.9023   16.3978   -8.4027   ...   -2.7940,
 'z': <pyarma.pyarma.cube object at 000001E392E6DD70>
 [cube size: 40x548x2]
 [cube slice: 0]
    0.5184  -0.0475  -1.0429   ...   0.7912
    1.3544  -0.3214   0.6103   ...  -0.0589
    0.5841   2.2018   0.5817   ...  -0.6557
         :        :        :   ...        :
    3.1964  -4.8537   5.1755   ...   6.8904
 
 [cube slice: 1]
    1.3339  -0.7138  -1.0815   ...  -1.4446
   